# FourD.js
[Joshua Marshall Moore](thwee.abacadabra.alchemist@gmail.com)
2017-11-18 CE

This notebook goes over FourD.js, offering small changes in the process, before I determine what can be ripped out and written more efficiently. 

## General Usage
This comment should describe general usage for the FourD class. 

In [ ]:
'use strict';
/*
 four-d.js
 Joshua M. Moore
 April 23, 2015

 depends on: mrdoob's three.js, rev66
 https://github.com/mrdoob/three.js/tree/r66
 tested with r66
 
 This is semantic version v0.1.0.
 See http://semver.org/
 
 The API works as follows:
 Every function or code block marked api is part of the API.
 
 Here's how it works:
 
 var fourd = new FourD(); // instantiation
 fourd.init('#selector', {width: 600, height: 350}); // initialization
 
 
 var vertex_options = {
   width: 5, // pixels
   height: 5,
   depth: 5,
   color: 0x000000 // hex color
 };
 
 var vertex_options = {
   cube: {
     size: 5,
     color: 0x000000 // or: texture: 'path_to.png'
   },
   label: {
     text: "Hello, WOrlds"
   }
 }
 
 // or:
 
 var vertex_options = {
   width: 5,
   height: 5,
   depth: 5,
   texture: 'path_to,png'
 }
 
 // or:
 
 // leave out vertex options altogether.
 // default values will be used.
 
 
 var vertex1 = fourd.graph.add_vertex(vertex_options); // add a vertex
 var vertex2 = fourd.graph.add_vertex(vertex_options); // add another vertex
 
 var edge_options = {}; // currently not defined, but this is how you would pass them.
 
 var edge = fourd.graph.add_edge(vertex1, vertex2, edge_options);
 
 fourd.graph.remove_edge(edge); // this is why you keep those variables
 fourd.graph.remove_vertex(vertex1);
 fourd.graph.remove_vertex(vertex2);
 
*/

We'll see...

It helps to break up the following code into chunks. Unfortunately, I wrapped everything in a FourD function.

In [ ]:
var FourD = function(){

FourD is a four dimensional graph visualization. Why four dimensional, you ask? Because because of its dynamic property, you can undo and redo steps if you apply the right pattern. Don't quiz me on Design Patterns right now, though. It's been nearly a decade, and a million things have happened since. 

In [ ]:
  var that = this;

This trick allows functions defined later on in the code to access properties and functions defined in FourD scope. 

In [ ]:
  var CONSTANTS = {
    width: 1000,
    attraction: 0.025,
    far: 1000,
    optimal_distance: 1.0,
    minimum_velocity: 0.001,
    friction: 0.60,
    zoom: -50,

    gravity: 0.070,

    rangeMinIn: -1.0,
    rangeMaxIn: 1.0,
    rangeMinOut: -10,
    rangeMaxOut: 10,

    BHN3: {
      inner_distance: 0.036,
      repulsion: 100.0,
      epsilon: 0.1
    }
  };
  

## The Vertex Class
I wonder whether the genetic algorithm I've been playing with could be used to vary the variables in this simulation. Gravity is useless by the way, just a pipedream for now. I know it's 9.81 m/s^2, but I don't know how to translate that into the acceleration function. Perhaps if I had a list of Factors influencing each Attribute. An example of a Factor would be gravity on acceleration. I guess it wouldn't matter if the camera was in free fall. These variables have been carefully tested to work on my machine. 

In [ ]:
  var is_vertex = function(potential){
    return potential.hasOwnProperty('id') && 
      potential.hasOwnProperty('edge_count') && 
      potential.hasOwnProperty('edges');
  };

I don't think I've ever used this function. A quick search should reveal that: I was wrong. Even though this function looks kinda convoluted, it looks like I use it to make sure the vertex has the necessary attributes to be used in the rest of the graph. I guess this could be useful in order to facilitate interoperability with other vertices, which makes this function sort of an interface, or a contract. 

In [ ]:
  /* 
    Vertex
    
    Creates a vertex which can be passed to Graph. 
    setting a label property in the options parameter can allow
    for a label to be drawn above or below a vertex. 
    
    Options:
    - invisible
    - see cube
    - see label
    
  */

That's just poor documentation. I'll work on something better. 

In [ ]:
  var Vertex = function(id, options){
    this.options = options;
    this.id = id;
    
    this.position = new THREE.Vector3(0, 0, 0);
    this.velocity = new THREE.Vector3(0, 0, 0);
    this.acceleration = new THREE.Vector3(0, 0, 0);
    
    this.toString = function(){
      return this.id.toString();
    };

    this.edge_count = 0;
    this.edges = {};
    this.neighbors = [];
    
    if(!this.options.hasOwnProperty('label')){
      this.options.label = {
        text: '',
        paint: false,
        direction: 'x',
        distance: '10'
      };
    };
  };

The Vertex is a busy class. It keeps track of its own position, velocity, and acceleration. The toString method looks well intentioned, and was probably useful for small outputs, but doesn't belong here. Instead, a collection of historical data would probably work. 

In [ ]:
  var getPicture = function(message){
    var canvas = document.createElement('canvas');
    var context = canvas.getContext('2d');
    var fontSize = 24;
    context.font = fontSize + "px Times New Roman";
    var measurements = context.measureText(message);
    //canvas.width = measurements.width + 10;
    //canvas.height = fontSize + 2;
    context.width = context.height = measurements.width + 10;
    context.fillText(message, 0, fontSize);
    
    return canvas;
  };

This function is used for the label functionality, as ill as it came out of my little bit forge. It's a neat little application of the HTML5 canvas, which can be used to create data urls which in turn feed src attributes. 

In [ ]:
  var getLabel = function(message){
    var texture = new THREE.Texture(getPicture(message));
    texture.minFilter = THREE.LinearFilter;
    texture.needsUpdate = true;
    var spriteMaterial = new THREE.SpriteMaterial({map: texture});
    var sprite = new THREE.Sprite(spriteMaterial);
    return sprite;
  };

The getLabel function returns a sprite, which should be an Object3D if I'm not mistaken. That means I was able to use it instead of a cube. 

## The Label Class 
The getLabel function should be called `Label`, plain and simple. like `Cube`. It's a sibling in ThreeJS's hierarchy. Or at least I'm trying to use it as such. 

In [ ]:
  var Label = function(options){
    var texture = new THREE.Texture(getPicture(text));
    texture.minFilter = THREE.LinearFilter; // ?
    texture.needsUpdate = true;
    var material = new THREE.SpriteMaterial({map: texture});
    var sprite = new THREE.Sprite(material);
    return sprite;
  }

This should look similar to the Cube function. Let's explore: 

In [ ]:
  Vertex.prototype.paint = function(scene){
    this.object = getLabel('HELLO, WORLDS!');
    scene.add(this.object);
  };

This function is obviously still a leftover from the days of the Label Experiments. They were semisuccessful, the closest I have come to labels yet, completely useless without object picking and effective camera control. This paint function could use an option to decide whether to draw a cube or a label. 

In [ ]:
  Vertex.prototype.paint = function(scene){
    if(this.options.cube){
      this.object = new Cube(this.options);
    }else if(this.options.label){
      this.object = new Label(this.options);
    }
    scene.add(this.object);
  }

This is a slightly revised version of `Vertex.prototype.paint(scene)` that distinguishes between a cube and a label. Not perfect but perhaps one day I'll get something like this to work: 

I found the following code snippet on the three js documentation website: 

In [ ]:
var geometry = new THREE.BoxBufferGeometry( 1, 1, 1 );
var material = new THREE.MeshBasicMaterial( {color: 0x00ff00} );

var cubeA = new THREE.Mesh( geometry, material );
cubeA.position.set( 100, 100, 0 );

var cubeB = new THREE.Mesh( geometry, material );
cubeB.position.set( -100, -100, 0 );

//create a group and add the two cubes
//These cubes can now be rotated / scaled etc as a group
var group = new THREE.Group();
group.add( cubeA );
group.add( cubeB );

scene.add( group );

In [ ]:
  Vertex.prototype.paint = function(scene){
    this.object = new THREE.Group();
    
    if(this.options.cube){
      var cube = new Cube(this.options.cube);
      cube.position.set(0, 0, 0);
      this.object.add(cube);
    }
    if(this.options.label){
      var label = new Label(this.options.label);
      label.position.set(0, this.options.cube ? this.options.cube.size*1.25 : 0, 0);
      this.object.add(cube);
    }
      
    scene.add(this.object);
  }
  
  // theoreitcally, this should work. 

This function should be able to group two objects, a mesh and a sprite into one group to be manipulated as one, and display them on the screen. The syntax is as follows:
`graph.add_vertex({cube: {size: 5, texture: src_or_data}, label: {text: 'Hello, Worlds!'}}`. This only requires text to be implemented, it's currently called message from earlier experiments. __TODO__

In [ ]:
  /* 
    Vertex.remove(...) 
    removes either a label: Vertex.remove('label'),
    or the vertex's cube: Vertex.remove('cube').
  */ 
  Vertex.prototype.remove = function(name){
    this.object.remove(name)
  }

It should be noted that the above function merely removes the object3d from the scene, and should hence be called in conjunction with the `Graph.remove_vertex(...)` function. 

## The Edge Class
The edge class is repsonsible for connecting two vertices, and holds a source and a target vertex. 

In [ ]:
  // Edge
  var Edge = function(id, source, target, options){
    this.options = options === undefined ? {strength: 1.0} : options;
    
    if(arguments.length < 3){
      throw new Error('Edge without sufficent arguments');
    }

    if(!is_vertex(source)){
      var source_type = typeof source;
      var source_error_msg = 'Source should be a Vertex instead of a ' + src_type + '.';
      throw new Error(src_error_msg);
    }

    if(!is_vertex(target)){
      var target_type = typeof target;
      var target_error_msg = 'Target should be a Vertex instead of a ' + tgt_type + '.';
      throw new Error(tgt_error_msg);
    }

    this.id = id;
    if(this.options.hasOwnProperty('directed')){
      this.gravity = true;
    }

    this.source = source;
    this.target = target;

    this.source.edge_count += 1;
    this.target.edge_count += 1;

    this.source.edges[this.id] = this;
    this.target.edges[this.id] = this;

    this.order = Math.random();
  };

The edge holds options. It's unclear as of right now whether strength actually has an effect on the rest of the code. This is one of those things a graph of code might be able to reveal at a glance. At a minimum, however, an edge consists of a source and a target vertex, and a random order that is used in the Barnes Hut Tree as a sorting property to get things started. The edge also registers itself with the source and target node via its own id, assigned through the constructor. This could probably be achieved statically. 

In [ ]:
  Edge.prototype.paint = function(scene){
    this.object = line(scene, this.source, this.target, this.options);
  };

Nothing to change here. I pass the options on to the paint function, where they may or may not get used. 

In [ ]:
  Edge.prototype.toString = function(){
    return this.source.toString() + '-->' + this.target.toString(); 
  };

Again, I don't know whether these toString functions get used in actual code. I don't think I've ever used them, but I'll keep them around for debugging. 

In [ ]:
  Edge.prototype.destroy = function(scene){
    delete this.source.edges[this.id];
    delete this.target.edges[this.id];

    CONSTANTS.scene.remove(this.object);
    delete this.object;
    
    this.source.edge_count--;
    this.target.edge_count--;
  };

This destroy function was carefully crafted to be run in the graph's clear function. It unregisters itself from its source and target vertices, and undraws itself from the scene. 

## The Graph
The graph holds vertices and edges, and is responsible for their proper addition and removal. 

In [ ]:
  // Graph
  var Graph = function(scene){
    this.scene = scene;
    this.type = 'Graph';
    this.vertex_id_spawn = 0;
    this.V = {};

    this.edge_id_spawn = 0;
    this.E = {};

    this.edge_counts = {};
  };

This graph knows how to paint itself via the scene constructor parameter. The `type` attribute is probably unneccessary. Other than that, a Graph is a collection of Vertices and Edges. The graph forms the primarily exposed API for manipulating the image produced. The comment above the `clear` method indicates that it's clearly meant to be exposed to the user of this library (and the end user, but that's another story).

In [ ]:
  // api
  Graph.prototype.clear = function(){

    for(var e in this.E){
      this.E[e].destroy(that.scene);
    }

    for(var v in this.V){
      // this.scene.remove...
      scene.remove(this.V[v].object);
      // this.V[v].destroy();
    }
    
    this.V = {};
    this.E = {};
    this.edge_counts = {};
    this.edge_id_spawn = 0;
    this.vertex_id_spawn = 0;
  };

This basically ensures that all objects created by the graph are removed from the scene. 

In [ ]:
  Graph.prototype._make_key = function(source, target){
    return '_' + source.toString() + '_' + target.toString();
  };

Not sure whether this is still important. Let's do a quick text search: Ah, yes. It's used to generate a key for a pair of vertices and edges. It's used to remove edges, though I can't won't claims as to this approach's efficiency.

In [ ]:
  // api
  Graph.prototype.add_vertex = function(options){
    var v = new Vertex(this.vertex_id_spawn++, options);
    v.paint(this.scene);
    this.V[v.id] = v;

    return v;
  };

This creates a vertex in the graph, paints the vertex, and saves it to the list of vertices. 

In [ ]:
  // api
  Graph.prototype.add_edge = function(source, target, options){
    var key = '_' + source.id + '_' + target.id;
    var edge;
    
    if(!this.edge_counts.hasOwnProperty(key)){
      edge = new Edge(this.edge_id_spawn++, source, target, options);
      this.E[edge.id] = edge;
      this.edge_counts[key] = 1;
    }else{
      this.edge_counts[key]++;
      for(var e in target.edges){
        for(var r in source.edges){
          if(e === r){
            return source.edges[r];
          }
        }
      }
    }
    
    edge.paint(this.scene);
    return edge;
  };

Same as the vertex, except for an edge. It should be noted that a single edge is allowed between vertices. If the edge_count is properly used, it can be used to imitate multiple edges and simply multiply the force exerted by the vertex on the edge by the multiplier. 

In [ ]:
  // api
  Graph.prototype.remove_edge = function(edge){
    var key = this._make_key(edge.source, edge.target);
    if(--this.edge_counts[key] === 0){
      edge.destroy();
      delete this.E[edge.id];
    }
  };

Removes an edge, simple stuff.

In [ ]:
  Graph.prototype.toString = function(){
    var edges = Object.keys(this.E).length;
    var nodes = Object.keys(this.V).length;

    return '|V|: ' + nodes.toString() + ',  |E|: ' + edges.toString();
  };

Another debugging function. Unlike the others for vertex and edge, this one doesn't get used in any critical make key functions yet. 

In [ ]:
  // api
  Graph.prototype.remove_vertex = function(vertex){
    for(var e in vertex.edges){
      vertex.edges[e].destroy(this.scene);  
      delete this.E[e];
    }
    
    this.scene.remove(vertex.object);
    delete this.V[vertex.id];
  };

Removes a vertex, simple stuff. 

In [ ]:
  var is_graph = function(potential){
    return potential.type === 'Graph';
  };

Why, I'm not sure this is needed at all! It's not like I'm doing graph addition or subtraction yet. 

## The Cube Class
The cube class is responsible for constructing a drawable cube, either with a texture or a color.

In [ ]:
  // apiish
  var Cube = function(scene, options){
    if(options === undefined){
      options = {};
    }
    
    if(options.width === undefined){
      options.width = 3;
    }
    if(options.height === undefined){
      options.height = 3;
    }
    if(options.depth === undefined){
      options.depth = 3;
    }
    if(options.color === undefined){
      options.color = 0x00ccaa;
    }
    
    if(options.wireframe === undefined){
      options.wireframe = false;
    }
    
    var geometry, material, material_args;
    geometry = new THREE.BoxGeometry(
      options.width,
      options.height,
      options.depth
    );
    geometry.dynamic = true;
    
    if(options.texture !== undefined){

      material_args = { 
        map: THREE.ImageUtils.loadTexture( options.texture )
      };
      
    }else{
      material_args = { 
        color: options.color,
        wireframe: true
      };
    }

    material = new THREE.MeshBasicMaterial( material_args );
    
    var cube = new THREE.Mesh( geometry, material );
    var scale = 2;
    cube.position = new THREE.Vector3(
      Math.random() * scale, Math.random() * scale,
      Math.random() * scale
    );
    cube.matrixAutoUpdate = true;
    
    scene.add(cube);
    return cube;
  };
  

The Cube function should more accurately be called `_Cube` and the Label function `_label`, since it's not part of the API. But other than that, it does what it's supposed to. It can be used where a label is currently used. The scene variable needs ot be yanked out of Cube, and left to paint. `size` is a sufficient cube parameter. 

In [ ]:
  // api-ish
  var Cube = function(options){
    if(options.color === undefined){
      options.color = 0x00ccaa;
    }
    
    if(options.wireframe === undefined){
      options.wireframe = false;
    }
    
    var geometry, material, material_args;
    geometry = new THREE.BoxGeometry(
      options.size,
      options.size,
      options.size
    );
    geometry.dynamic = true;
    
    if(options.texture !== undefined){
      material_args = {map: THREE.ImageUtils.loadTexture( options.texture )};
    }else{
      material_args = {color: options.color, wireframe: true};
    }

    material = new THREE.MeshBasicMaterial( material_args );
    
    var cube = new THREE.Mesh( geometry, material );
    //var scale = 2;
    cube.position = new THREE.Vector3(
      Math.random(),// * scale, 
      Math.random(),// * scale,
      Math.random() * scale
    );
    cube.matrixAutoUpdate = true;
    
    scene.add(cube);
    return cube;
  };  

That looks better. 

In [ ]:
  /*
    todo: a streamlined system for extracting options. 
    
    specifically:
      vertex_options.label and edge_options.label should be
      the same, and the label option should not conflict with 
      vertex- or edge- options.
      
    This system could be called an option extractor. I see a 
    stream of options being passed through the vertex and edge
    constructors and painters that takes the appropriate parts 
    of the options and applies them to the correct part in the 
    taxonomy of objects, i.e. vertex or label, or edge or label.
  */
  
  /*
    THREE.Sprite Label(parameters)
  
    label
    creates a label that can be added to a vertex or an edge.
    
    parameters
    - text
    - font_family      // optional: 'Arial'
    - font_size        // optional: 12
    - sprite_alignment // optional: top left
    
    returns
    a sprite
  */
  

Some of this is outdated. One thing on my todo list is definitely to be able to add a label beneath a vertex. 

## Another Label

In [ ]:
  var Label = function( parameters ){
    if( parameters === undefined ){
      parameters = {};
    }
    
    if(!parameters.text === undefined){
      throw "Can't do a label without some text";
    }
    
    var font_family = parameters.hasOwnProperty('font_family') ? 
      parameters['font_family'] : 'Arial';
    var font_size = parameters.hasOwnProperty('font_size') ? 
      parameters['font_size'] : 12;
    // border thickness
    // border color
    // background color
    
    var canvas = document.createElement('canvas');
    var context = canvas.getContext('2d');
    context.font = "Bold " + font_size + "px " + font_family;
    
    // get size data (height depends only on font size)
    var metrics = context.measureText(parameters.text);
    var text_width = metrics.width;
    //context.fillStyle = "rgba(0,0,0,0)";
    //context.strokeStyle = "rgba(0,0,0,0)";
    
    //context.lineWidth = 0;
    context.fillStyle = "rgba(0, 0, 0, 1.0)";
    context.fillText(parameters.text ? parameters.text : '', 1, font_size);
    
    var texture = new THREE.Texture(canvas);
    texture.needsUpdate = true;
    
    var sprite_material = new THREE.SpriteMaterial(
      {
        map: texture, 
        useScreenCoordinates: false
      }
    );
    
    var sprite = new THREE.Sprite( sprite_material );
    sprite.scale.set(100, 50, 1.0);

    return sprite;
  };

Again, I think this code is outdated, and the label code at the top should be used to draw labels. I shall investigate: Yes, this is now a duplicate. I shall keep it around until I know the other one works better. 

## The line function
The line function is used to draw edges. 

In [ ]:
  // apiish this will change
  // todo: make line options like cube options
  var line = function(scene, source, target){
    var geometry = new THREE.Geometry();
    geometry.dynamic = true;
    geometry.vertices.push(source.object.position);
    geometry.vertices.push(target.object.position);
    geometry.verticesNeedUpdate = true;
    
    var material = new THREE.LineBasicMaterial({ color: 0x000000 });
    
    var line = new THREE.Line( geometry, material );
      
    scene.add(line);
    return line;
  };

## The Barnes Hut Node 3
The Barnes Hute Node 3 (BHN3 for short) represents, in my opinion, the biggest achievement in Veldhuizen's paper. 

Here's where things get a little complicated, and there might be room for improvement, for example by converting the graph to matrix form. 

But first, let me investigate and describe what these functions do. 

In [ ]:
  var BHN3 = function(){
    this.inners = [];
    this.outers = {};
    this.center_sum = new THREE.Vector3(0, 0, 0);
    this.center_count = 0;
  };

The Barnes Hut Node 3 is a seven way tree that stores vertices closer than the constant `inner_distance` in the inners array, while storing everything outside that radius in one of the outers. There are six outers. I bet this class could be enhanced to keep track of the outermost vertex. 

In [ ]:
  var BHN3 = function(){
    this.inners = [];
    this.outers = {};
    this.center_sum = new THREE.Vector3(0, 0, 0);
    this.center_count = 0;
    this.furthest = 0;
  }

That looks a little better. I might revisit this code later and add the graph to the constructor. 

In [ ]:
  BHN3.prototype.constants = CONSTANTS.BHN3;

Some housekeeping. 

In [ ]:
  BHN3.prototype.center = function(){
    return this.center_sum.clone().divideScalar(this.center_count);
  };

This function estimates the center of the graph. 

In [ ]:
  BHN3.prototype.place_inner = function(vertex){
    this.inners.push(vertex);
    this.center_sum.add(vertex.object.position);
    this.center_count += 1;
  };

This function is used to add a vertex to the `inner` array. Perhaps if I could find a way of maintaining the tree instead of recreating it from scratch with every draw loop. 

In [ ]:
  BHN3.prototype.get_octant = function(position){
    var center = this.center();
    var x = center.x < position.x ? 'l' : 'r';
    var y = center.y < position.y ? 'u' : 'd';
    var z = center.z < position.z ? 'i' : 'o';
    return x + y + z;
  };

This function is used to determine which - 2 options each and there's 3 of them - eight octants the vertex's position will place the vertex in. 

In [ ]:
  BHN3.prototype.place_outer = function(vertex){
    var octant = this.get_octant(vertex.object.position);
    this.outers[octant] = this.outers[octant] || new BHN3();
    this.outers[octant].insert(vertex);
  };

`place_outer` takes the vertex, determines which octant (in a 2d map it'd be a quadrant, in a binary search tree, it would be a side) the vertex belongs in. This looks like two good opportunities: 

* updating `furthest`, and 
* looking up how to maintain a binary search tree

Those techniques could probably be used for the BHN3. 

In [ ]:
  BHN3.prototype.place_outer = function(vertex){
    var octant = this.get_octant(vertex.object.position);
    this.outers[octant] = this.outers[octant] || new BHN3();
    this.outers[octant].insert(vertex);
    
    var distance_from_center = vertex.object.position.distanceTo(this.center);
    if(distance_from_center > this.furthest){ // can i get away with not calling center again?
      
    } 
  };

In [ ]:
  BHN3.prototype.insert = function(vertex){
    if(this.inners.length === 0){
      this.place_inner(vertex);
    }else{
      if(this.center().distanceTo(vertex.object.position) <= this.constants.inner_distance){
        this.place_inner(vertex);
      }else{
        this.place_outer(vertex);
      }
    }
  };

Insert defers to place_inner or place_outer depending on how far a vertex is from the node's center.

In [ ]:
  BHN3.prototype.estimate = function(vertex, force, force_fn){
    if(this.inners.indexOf(vertex) > -1){
      for(var i=0; i<this.inners.length; i++){
        if(vertex !== this.inners[i]){
          var individual_force = force_fn(
            vertex.object.position.clone(),
            this.inners[i].object.position.clone()
          );
	  
          force.add(individual_force);
        }
      }
    }else{
      var sumstimate = force_fn(vertex.object.position, this.center());
      sumstimate.multiplyScalar(this.center_count);
      force.add(sumstimate);
    }
    
    for(var octant in this.outers){
      this.outers[octant].estimate(vertex, force, force_fn);
    }
  };

BHN3's estimate function can be used with the pairwise repulsion function. It's meant for situations where you obtain a value by traversing the entire or condensed tree. 

In [ ]:
  BHN3.prototype.pairwise_repulsion = function( x1, x2 ){

    var enumerator1, denominator1, 
      enumerator2, denominator2, 
      repulsion_constant, 
      difference, absolute_difference, 
      epsilon, product, 
      term1, term2,
      square, sum, result; 
    
    // first term
    enumerator1 = repulsion_constant = CONSTANTS.BHN3.repulsion;
    
    difference = x1.clone().sub(x2.clone());
    absolute_difference = difference.length();
    
    epsilon = CONSTANTS.BHN3.epsilon;
    sum = epsilon + absolute_difference;
    denominator1 = square = sum*sum;
    
    term1 = enumerator1 / denominator1;
    
    // second term
    enumerator2 = difference;
    denominator2 = absolute_difference;
    
    term2 = enumerator2.divideScalar(denominator2);
    
    // result
    result = term2.multiplyScalar(term1);  
    
    return result;
  };
  

The pairwise repulsion function is used with the estimate function in determining the forces acting on the vertices, and their edges.

## Graph.layout()
This function gets called at each animation frame. It is unfortunate that it is so complicated. 

In [ ]:
  var edges = false;
  Graph.prototype.layout = function(){

    // calculate repulsions
    var tree = new BHN3();
    var vertex, edge, v, e;
    
    for(v in this.V){
      vertex = this.V[v];
      vertex.acceleration = new THREE.Vector3(0.0, 0.0, 0.0);
      vertex.repulsion_forces = new THREE.Vector3(0.0, 0.0, 0.0);
      vertex.attraction_forces = new THREE.Vector3(0.0, 0.0, 0.0);

      tree.insert(vertex);
    }
    
    for(v in this.V){
      vertex = this.V[v];
      vertex.repulsion_forces = vertex.repulsion_forces || new THREE.Vector3();
      vertex.repulsion_forces.set(0.0, 0.0, 0.0);
      tree.estimate(
        vertex, vertex.repulsion_forces,
        BHN3.prototype.pairwise_repulsion
      );
    }
    
    // calculate attractions
    
    for(e in this.E){
      edge = this.E[e];
      
      var attraction = edge.source.object.position.clone().sub(
        edge.target.object.position
      );
      attraction.multiplyScalar(-1 * CONSTANTS.attraction);

      // attraction.multiplyScalar(edge.options.strength);

      edge.source.attraction_forces.sub(attraction);
      edge.target.attraction_forces.add(attraction);

      if(edge.gravity){
        var gravity = new THREE.Vector3(0.0, -1 * CONSTANTS.gravity, 0.0);
        edge.target.acceleration.add(gravity);
      }
    }
    
    for(v in this.V){
      // update velocity
      vertex = this.V[v];
      if(vertex){
        var friction = vertex.velocity.multiplyScalar(CONSTANTS.friction);

        vertex.acceleration.add(
          vertex.repulsion_forces.clone().add(
            vertex.attraction_forces.clone().negate()
          )
        );
        vertex.acceleration.sub(friction);
        
        vertex.velocity.add(vertex.acceleration);
        vertex.object.position.add(vertex.velocity);
      }
    }
    
    for(e in this.E){
      edge = this.E[e];

      if(edge){  
        edge.object.geometry.dirty = true;
        edge.object.geometry.__dirty = true;
        edge.object.geometry.verticesNeedUpdate = true;
      }
    }

    this.center = tree.center();
  };

This is the layout function in all its painstaking glory. There are four loops in here, all achieving differnet things. Maybe some of those things can overlap. Maybe the edge attraction math can be done in a two dimensional array. 

In [ ]:
  var edges = false;
  Graph.prototype.layout = function(){

    // calculate repulsions
    var tree = new BHN3();
    var vertex, edge, v, e;
    
      

So far we've declared some commonly used variables. 

In [ ]:
      
    for(v in this.V){
      vertex = this.V[v];
      vertex.acceleration = new THREE.Vector3(0.0, 0.0, 0.0);
      vertex.repulsion_forces = new THREE.Vector3(0.0, 0.0, 0.0);
      vertex.attraction_forces = new THREE.Vector3(0.0, 0.0, 0.0);

      tree.insert(vertex);
    }
    

This loops makes sure every vertex has an `acceleration`, `repulsion_forces` and `attraction_forces` before inserting the vertex into the tree initialized earlier. It would probably be of great benefit to be able to maintain the tree across iterations. 

In [ ]:
    for(v in this.V){
      vertex = this.V[v];
      vertex.repulsion_forces = vertex.repulsion_forces || new THREE.Vector3();
      vertex.repulsion_forces.set(0.0, 0.0, 0.0);
      tree.estimate(
        vertex, vertex.repulsion_forces,
        BHN3.prototype.pairwise_repulsion
      );
    }

This loop over the vertices assigns each vertex the property of repulsion force, which could probably be done upon vertex initialization, and estimates these repulsion forces using the pairwise repulsion function defined earlier. 

In [ ]:
    // calculate attractions
    
    for(e in this.E){
      edge = this.E[e];
      
      var attraction = edge.source.object.position.clone().sub(
        edge.target.object.position
      );
      attraction.multiplyScalar(-1 * CONSTANTS.attraction);

      // attraction.multiplyScalar(edge.options.strength);

      edge.source.attraction_forces.sub(attraction);
      edge.target.attraction_forces.add(attraction);

      if(edge.gravity){
        var gravity = new THREE.Vector3(0.0, -1 * CONSTANTS.gravity, 0.0);
        edge.target.acceleration.add(gravity);
      }
    }

This loop over the edges assigns positive and negative attraction to each of the connected vertices. 

In [ ]:
    for(v in this.V){
      // update velocity
      vertex = this.V[v];
      if(vertex){
        var friction = vertex.velocity.multiplyScalar(CONSTANTS.friction);

        vertex.acceleration.add(
          vertex.repulsion_forces.clone().add(
            vertex.attraction_forces.clone().negate()
          )
        );
        vertex.acceleration.sub(friction);
        
        vertex.velocity.add(vertex.acceleration);
        vertex.object.position.add(vertex.velocity);
      }
    }

This loop over the vertices applies friction to the repulsion.forces and subtracts/adds repulsion and attraction forces from the vertexes acceleration before "downgrading" to velocity and ultimately, position. 

In [ ]:
    for(e in this.E){
      edge = this.E[e];

      if(edge){  
        edge.object.geometry.dirty = true;
        edge.object.geometry.__dirty = true;
        edge.object.geometry.verticesNeedUpdate = true;
      }
    }

In this final loop, the edge geometries are updated so they can be redrawn on the screen. 

In [ ]:
    this.center = tree.center();
  };

Finally, we recompute the center of the graph.

## Housekeeping, setup and the render loop

In [ ]:
  this._internals = {};
  var scene,
      element,
      camera,
      light,
      renderer,
      graph;

  var render = function render(){
    requestAnimationFrame(render);
    graph.layout();
    renderer.render(scene, camera);
    camera.position.z = graph.center.z - CONSTANTS.zoom;
    camera.lookAt(graph.center);
  };

  var clear = function clear(){
    graph.clear();
  };
  
  // api
  this.init = function(selector, options){
    scene = new THREE.Scene();
    element = document.querySelector(selector);
    if(!element){
      throw "element " + selector + " wasn't found on the page.";
    }
    camera = new THREE.PerspectiveCamera(
      70,
      options.width / options.height,
      1,
      CONSTANTS.far
    );
    light = new THREE.PointLight( 0xeeeeee ); // soft white light
    
    CONSTANTS.scene = scene;
    scene.add( camera );
    scene.add( light );
    
    renderer = new THREE.WebGLRenderer();
    renderer.setClearColor(0xefefef);
    renderer.setSize( options.width, options.height );
    
    document.querySelector(selector).appendChild( renderer.domElement );
    
    graph = new Graph(scene);
    
    camera.position.z = -250;
    camera.lookAt(new THREE.Vector3(0, 0, 0));

    that._internals = {
      scene: scene,
      element: element,
      camera: camera,
      light: light,
      renderer: renderer,
      graph: graph
    };

    // api
    this.version = "0.1.0";
    this.graph = graph;
    this.render = render;
    this.clear = clear;
    this.variables = CONSTANTS;

    render();
  };
  
  this._internals = {
    Vertex: Vertex,
    Edge: Edge,
    Graph: Graph,
    BHN3: BHN3,
  };

  // untested
  this.setCubeFn = function(fn){
    cube = fn;
  };

  this.setLineFn = function(fn){
    line = fn;
  };
  // end untested
  
  return this;
};

## No Success: JSDOM + THREE.JS

The goal of the undertakings below is to run something that belongs in a browser in an environment that unfortunately does not offer an iframe. 

In [ ]:
var jsdom = require("jsdom");
global.document = jsdom.jsdom();

This doesn't seem to be working. I'll have to ask about how to install jsdom in jupyter. 

I tried installing `npm -g install require` just to be sure. Let's try again!

In [ ]:
var jsdom = require("jsdom");

Maybe running `npm install -g jsdom` in the Command Prompt will work since I'm on Windows. I tried Cygwin64 earlier. 

In [ ]:
var jsdom = require("jsdom");

Yippie!

Back to my javascript example from [stackoverflow](https://stackoverflow.com/questions/43488464/how-to-output-svg-in-a-jupyter-notebook-using-jsdom-d3-and-ijavascript/43514776#43514776) via [github](https://github.com/n-riesco/ijavascript/issues/120). I found some documentation at https://github.com/tmpvar/jsdom, but here is one of the cool things about javascript and python:

In [ ]:
console.log(jsdom);

From that, I can see that the jsdom object has a JSDOM function that I can try calling...

In [ ]:
var my_dom = new jsdom.JSDOM()

That worked. Let's see what's in it: 

In [ ]:
console.log(my_dom);

A JSDOM, how helpful..., but https://github.com/tmpvar/jsdom#jsdom-object-api to the rescue, let's see if we can't crack this nut. 

In [ ]:
console.log(my_dom.window)

In [ ]:
my_dom.serialize()

In [ ]:
my_dom

In [ ]:
jsdom.JSDOM.fromURL('https://thwee-alchemist.github.io/fourd.js')

In [ ]:
var mydom = jsdom.JSDOM.fromURL('https://thwee-alchemist.github.io/fourd.js/')
jsdom

In [ ]:
fourd_console = new jsdom.VirtualConsole();